In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append("..")
import utility.utility_functions as ut

In [2]:
df = ut.load_post_data_prep_all_zone()

In [3]:
df.dropna(inplace=True)
df = df.drop(df[df.trip_time < 5].index)
df = df.drop('pickup_date',axis=1)
df.reset_index(inplace=True)

In [4]:
main_df = ut.get_dummies(['weekday','time_binned'],df)

In [5]:
main_df = ut.swap_first_and_last_col(main_df)

In [6]:
main_df.head()

,Zone,00:00 - 00:59,01:00 - 01:59,02:00 - 02:59,03:00 - 03:59,04:00 - 04:59,05:00 - 05:59,06:00 - 06:59,07:00 - 07:59,08:00 - 08:59,...,Tdep,HDD,CDD,Precipitation,new_snow,snow_depth,trip_distance,trip_time,speed,num_of_taxis
0,4,1,0,0,0,0,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.543171,11.890650,12.167499,41
1,4,0,1,0,0,0,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.842436,14.737821,11.223936,78
2,4,0,0,1,0,0,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.235484,11.183333,11.346154,62
3,4,0,0,0,1,0,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.640308,11.701795,12.651760,65
4,4,0,0,0,0,1,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.635000,11.158889,13.186688,30


In [7]:
y, X = ut.split_X_y(main_df,'num_of_taxis')